In [58]:
import requests  # API requests
import pandas as pd
import xml.etree.ElementTree as ET  # Will be used to parse the data

hi


In [48]:
# Link to connect to the API
api_link = "http://reports.sem-o.com/api/v1/documents/static-reports"

In [49]:
def apiQuery(parameters):
    Response = requests.get(api_link, params=parameters)
    return Response

In [50]:
# Specifying our parameters
startDate = '2024-05-20'
endDate = '2024-05-26'
PageSize = '1'
SortBy = 'PublishTime'
ForecastReportName = 'Forecast Availability'
OutturnReportName = 'Average Outturn Availability'
ResourceName = ''
ParticipantName = 'PT_400116'

# Getting the range of dates
dateRange = pd.date_range(start=startDate, end=endDate).date

In [51]:
# Function to filter XML strings from a JSON object
def filterXMLStrings(json_data):
    try:
        items = json_data.get('items', [])
        filtered_strings = [item['ResourceName'] for item in items if item['ResourceName'].endswith('.xml')]
        return filtered_strings
    except KeyError:
        print("KeyError: The JSON object does not contain the expected keys.")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []


In [72]:
# Function to loop through dates and return all the XML file names we need
def RetrieveXMLFileNames(XMLFileType):
    XMLFileNames = []
    for date in dateRange:
        TypeParameters = {
            'ReportName': XMLFileType,
            'Date': date,
            'page_size': PageSize,
            'sort_by': SortBy,
        }
        Response = apiQuery(TypeParameters)
        if Response.status_code != 200:
            print(f"Failed to retrieve data: {Response.status_code}")
        else:
            GridInfo = Response.json()

            # Filtering data to isolate the names of the XML files we want to use
            FilteredGridInfo = filterXMLStrings(GridInfo)
            XMLFileNames.extend(FilteredGridInfo)  # Extend instead of append

            #print("\n\n\n" + XMLFileType + ":", FilteredGridInfo)

    return XMLFileNames


In [84]:
ForecastResponseData = RetrieveXMLFileNames(ForecastReportName)
OutturnResponseData = RetrieveXMLFileNames(OutturnReportName)

print(ForecastResponseData)

forecast_availability_array = []

for data in ForecastResponseData:
    url = 'https://reports.sem-o.com/documents/' + data
    df = pd.read_xml(url)
    
if 'ParticipantName' in df.columns and 'ForecastAvailability' in df.columns:
        filtered_df = df[df['ParticipantName'] == ParticipantName]
        forecast_availability_array.extend(filtered_df['ForecastAvailability'].dropna().tolist())

print("Forecast Availability:", forecast_availability_array)




print(OutturnResponseData)

outturn_availability_array = []

for data in OutturnResponseData:
    url = 'https://reports.sem-o.com/documents/' + data
    df = pd.read_xml(url)
    
if 'ParticipantName' in df.columns and 'AvgOutturnAvail' in df.columns:
        filtered_df = df[df['ParticipantName'] == ParticipantName]
        outturn_availability_array.extend(filtered_df['OutturnAvailability'].dropna().tolist())

print("Outturn Availability:", outturn_availability_array)



['PUB_DailyForecastAvailability_202405192030.xml', 'PUB_DailyForecastAvailability_202405202030.xml', 'PUB_DailyForecastAvailability_202405212030.xml', 'PUB_DailyForecastAvailability_202405222030.xml', 'PUB_DailyForecastAvailability_202405232030.xml', 'PUB_DailyForecastAvailability_202405242030.xml', 'PUB_DailyForecastAvailability_202405252030.xml']
Forecast Availability: [8.924, 8.924, 8.734, 8.734, 8.68, 8.68, 8.652, 8.652, 8.617, 8.617, 8.845, 8.845, 8.781, 8.781, 8.18, 8.18, 7.918, 7.918, 7.886, 7.886, 7.794, 7.794, 7.739, 7.739, 7.569, 7.569, 7.294, 7.294, 7.418, 7.418, 7.454, 7.454, 7.416, 7.416, 7.421, 7.421, 7.577, 7.577, 7.786, 7.786, 7.831, 7.831, 8.003, 8.003, 9.043, 9.043, 9.097, 9.097, 6.519, 6.519, 6.517, 6.517, 6.523, 6.523, 6.518, 6.518, 6.528, 6.528, 8.348, 8.348, 8.364, 8.364, 8.361, 8.361, 8.371, 8.371, 8.357, 8.357, 8.322, 8.322, 8.288, 8.288, 8.288, 8.288, 8.292, 8.292, 8.304, 8.304, 8.3, 8.3, 8.053, 8.053, 8.075, 8.075, 8.3, 8.3, 8.315, 8.315, 8.304, 8.304, 6.414, 